In [8]:
# Import the modules
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import tqdm
import statsmodels.api as sm

In [7]:
# Import Data
df_monthly = pd.read_excel('PredictorData2023.xlsx',sheet_name="Monthly")
# Parse the dates properly
time = [str(d) for d in df_monthly.yyyymm]
df_monthly.index = pd.to_datetime(time,format="%Y%m")

In [39]:
# Variable construction
df_monthly['ExRet'] = df_monthly['CRSP_SPvw']-df_monthly['Rfree']
df_monthly['DP'] = np.log(df_monthly['D12'])-np.log(df_monthly['Index'])
df_monthly['DY'] = np.log(df_monthly['D12'])-np.log(df_monthly['Index'].shift())
df_monthly['EP'] = np.log(df_monthly['E12'])-np.log(df_monthly['Index'])
df_monthly['DE'] = np.log(df_monthly['D12'])-np.log(df_monthly['E12'])
df_monthly['tms'] = df_monthly['lty']-df_monthly['tbl']
df_monthly['dfr'] = df_monthly['corpr']-df_monthly['ltr']
df_monthly['dfy'] = df_monthly['BAA']-df_monthly['AAA']

# infl needs to be lagged one more month
df_monthly['infl'] = df_monthly['infl'].shift().copy()

# Construction of dependent and independent variables
dep_var = 'ExRet'
indep_vars = ['DE','svar','dfr','lty','ltr','infl','tms','tbl','dfy','DP','DY','EP','b/m','ntis']

# Use the data from 1926/12 to 2023/12
subperiod = df_monthly.index>='1926-12-01'
df = df_monthly[subperiod]
M = 240 # Initial length of estimation window
gam = 3 # risk aversion coefficient

# Create the benchmark using historical average
Hist_Mean = np.asarray(df[dep_var].expanding().mean().shift())
Hist_Variance = np.asarray(df[dep_var].expanding().var().shift())

# Benchmark SSE (Historical Average)
OOS_SSE_Hist = np.sum((df[dep_var][M+1:]-Hist_Mean[M+1:])**2)

# Benchmark Certainty Equivalence
w0 = ((1/gam)*(Hist_Mean/Hist_Variance)).clip(None,1.5);
r0 = df[dep_var]*w0
CE_Hist = np.mean(r0[M+1:])-gam/2*np.var(r0[M+1:],ddof=1)

### The following codes demonstrate how to compute OOS $R^2$ and CEV for one predictive regression (using DY)

In [40]:
Y = np.asarray(df[dep_var])
X = np.asarray(df['DY'])
Y_Hat = np.full(len(Y), np.nan)
X = sm.add_constant(X)
# Note that we start the index at M+1 because the first element of predicted return is at t=M+2.
for i in range(M+1,len(Y)):
    Y1 = Y[1:i]
    X1 = X[0:i-1,:] 
    reg = sm.OLS(Y1, X1, missing='drop').fit()
    Y_Hat[i] = np.take(reg.predict(X[i-1,:]),0)       # The predicted value is based on the observation before


In [41]:
OOS_SSE = np.sum((Y[M+1:]-Y_Hat[M+1:])**2)
OOS_R2 = 1-OOS_SSE/OOS_SSE_Hist
w1 = ((1/gam)*(Y_Hat/Hist_Variance)).clip(None,1.5);
r1 = Y*w1
CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:],ddof=1)

### In-sample $R^2$ and out-of-sample $R^2$

In [42]:
reg1 = sm.OLS(Y[1:],X[0:len(Y)-1,:],missing='drop').fit()
IS_R2 = reg1.rsquared
print("IS R^2 = %6.3f"%(100.0*IS_R2))
print("OOS R^2 = %6.3f"%(100.0*OOS_R2))

IS R^2 =  0.359
OOS R^2 = -0.973


### $\Delta CEV$

In [43]:
print('Difference in Certainty Equivalence = %7.4f'%(100*(CE-CE_Hist)))

Difference in Certainty Equivalence = -0.1558


### OOS $R^2$ and CEV for all independent variables

In [44]:
for variable in indep_vars:
    Y = np.asarray(df[dep_var])
    X = np.asarray(df[variable])
    Y_hat = np.full(len(Y), np.nan)
    X = sm.add_constant(X)

    for i in range(M+1, len(Y)):
        Y1 = Y[1:i]
        X1 = X[0:i-1, :]
        reg = sm.OLS(Y1, X1, missing='drop').fit()
        Y_hat[i] = np.take(reg.predict(X[i-1, :]), 0)

    OOS_SSE = np.sum((Y[M+1:]-Y_hat[M+1:])**2)
    OOS_R2 = 1 - OOS_SSE/OOS_SSE_Hist
    w1 = ((1/gam)*(Y_hat/Hist_Variance)).clip(None, 1.5);
    r1 = Y*w1
    CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:], ddof=1)

    reg1 = sm.OLS(Y[1:], X[0:len(Y)-1, :], missing='drop').fit()
    IS_R2 = reg1.rsquared
    print(f"{variable} IS R^2 = %6.3f"%(100.0*IS_R2))
    print(f"{variable} OOS R^2 = %6.3f"%(100.0*OOS_R2))

    print(f'{variable} Difference in Certainty Equivalence = %7.4f'%(100*(CE-CE_Hist)))

DE IS R^2 =  0.005
DE OOS R^2 = -0.654
DE Difference in Certainty Equivalence =  0.0051
svar IS R^2 =  0.019
svar OOS R^2 = -0.608
svar Difference in Certainty Equivalence = -0.0535
dfr IS R^2 =  0.064
dfr OOS R^2 = -0.341
dfr Difference in Certainty Equivalence = -0.0135
lty IS R^2 =  0.197
lty OOS R^2 = -0.271
lty Difference in Certainty Equivalence = -0.0715
ltr IS R^2 =  0.203
ltr OOS R^2 = -0.367
ltr Difference in Certainty Equivalence =  0.0003
infl IS R^2 =  0.138
infl OOS R^2 = -0.026
infl Difference in Certainty Equivalence = -0.0103
tms IS R^2 =  0.129
tms OOS R^2 =  0.050
tms Difference in Certainty Equivalence =  0.0419
tbl IS R^2 =  0.315
tbl OOS R^2 =  0.240
tbl Difference in Certainty Equivalence = -0.0123
dfy IS R^2 =  0.254
dfy OOS R^2 = -0.284
dfy Difference in Certainty Equivalence = -0.0547
DP IS R^2 =  0.275
DP OOS R^2 = -0.341
DP Difference in Certainty Equivalence = -0.1226
DY IS R^2 =  0.359
DY OOS R^2 = -0.973
DY Difference in Certainty Equivalence = -0.1558
EP

### Kitchen sink Regression $R^2$ and CEV

In [45]:
kitchen_vars = ['svar','dfr','lty','ltr','infl','tbl','dfy','DP','DY','EP','b/m','ntis']

Y = np.asarray(df[dep_var])
X = np.asarray(df[kitchen_vars])
Y_hat = np.full(len(Y), np.nan)
X = sm.add_constant(X)

for i in range(M+1, len(Y)):
    Y1 = Y[1:i]
    X1 = X[0:i-1, :]
    reg = sm.OLS(Y1, X1, missing='drop').fit()
    Y_hat[i] = np.take(reg.predict(X[i-1, :]), 0)

OOS_SSE = np.sum((Y[M+1:]-Y_hat[M+1:])**2)
OOS_R2 = 1 - OOS_SSE/OOS_SSE_Hist
w1 = ((1/gam)*(Y_hat/Hist_Variance)).clip(None, 1.5);
r1 = Y*w1
CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:], ddof=1)

reg1 = sm.OLS(Y[1:], X[0:len(Y)-1, :], missing='drop').fit()
IS_R2 = reg1.rsquared
print("Kitchen Sink IS R^2 = %6.3f"%(100*IS_R2))
print("Kitchen Sink OOS R^2 = %6.3f"%(100*OOS_R2))

print('Kitchen Sink Difference in Certainty Equivalence = %7.4f'%(100*(CE-CE_Hist)))


Kitchen Sink IS R^2 =  2.675
Kitchen Sink OOS R^2 = -12.361
Kitchen Sink Difference in Certainty Equivalence = -0.5757


### Question 2: Forcing the predicted market risk premium to be nonnegative (OOS $R^2$ and $\Delta CEV$)

In [46]:
for variable in indep_vars:
    Y = np.asarray(df[dep_var])
    X = np.asarray(df[variable])
    Y_hat = np.full(len(Y), np.nan)
    X = sm.add_constant(X)

    for i in range(M+1, len(Y)):
        Y1 = Y[1:i]
        X1 = X[0:i-1, :]
        reg = sm.OLS(Y1, X1, missing='drop').fit()
        Y_hat[i] = np.take(reg.predict(X[i-1, :]), 0)

    OOS_SSE = np.sum((Y[M+1:]-Y_hat[M+1:])**2)
    OOS_R2 = 1 - OOS_SSE/OOS_SSE_Hist
    w1 = ((1/gam)*(Y_hat/Hist_Variance)).clip(0, 1.5) # Ensuring that weight are nonnegative so we have no short positions 
    r1 = Y*w1                                         # and we have nonnegative risk premiums
    CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:], ddof=1)

    reg1 = sm.OLS(Y[1:], X[0:len(Y)-1, :], missing='drop').fit()
    IS_R2 = reg1.rsquared
    print(f"{variable} OOS R^2 = %6.3f"%(100.0*OOS_R2))

    print(f'{variable} Difference in Certainty Equivalence = %7.4f'%(100*(CE-CE_Hist)))

DE OOS R^2 = -0.654
DE Difference in Certainty Equivalence =  0.0074
svar OOS R^2 = -0.608
svar Difference in Certainty Equivalence = -0.0525
dfr OOS R^2 = -0.341
dfr Difference in Certainty Equivalence = -0.0017
lty OOS R^2 = -0.271
lty Difference in Certainty Equivalence = -0.0132
ltr OOS R^2 = -0.367
ltr Difference in Certainty Equivalence =  0.0012
infl OOS R^2 = -0.026
infl Difference in Certainty Equivalence = -0.0088
tms OOS R^2 =  0.050
tms Difference in Certainty Equivalence =  0.0425
tbl OOS R^2 =  0.240
tbl Difference in Certainty Equivalence = -0.0018
dfy OOS R^2 = -0.284
dfy Difference in Certainty Equivalence = -0.0542
DP OOS R^2 = -0.341
DP Difference in Certainty Equivalence = -0.0945
DY OOS R^2 = -0.973
DY Difference in Certainty Equivalence = -0.1013
EP OOS R^2 = -1.539
EP Difference in Certainty Equivalence = -0.0292
b/m OOS R^2 = -2.208
b/m Difference in Certainty Equivalence = -0.1710
ntis OOS R^2 = -0.721
ntis Difference in Certainty Equivalence =  0.0439


### Kitchen Sink with Nonnegative Risk Premiums

In [47]:
kitchen_vars = ['svar','dfr','lty','ltr','infl','tbl','dfy','DP','DY','EP','b/m','ntis']

Y = np.asarray(df[dep_var])
X = np.asarray(df[kitchen_vars])
Y_hat = np.full(len(Y), np.nan)
X = sm.add_constant(X)

for i in range(M+1, len(Y)):
    Y1 = Y[1:i]
    X1 = X[0:i-1, :]
    reg = sm.OLS(Y1, X1, missing='drop').fit()
    Y_hat[i] = np.take(reg.predict(X[i-1, :]), 0)

OOS_SSE = np.sum((Y[M+1:]-Y_hat[M+1:])**2)
OOS_R2 = 1 - OOS_SSE/OOS_SSE_Hist
w1 = ((1/gam)*(Y_hat/Hist_Variance)).clip(0, 1.5);
r1 = Y*w1
CE = np.mean(r1[M+1:])-gam/2*np.var(r1[M+1:], ddof=1)

reg1 = sm.OLS(Y[1:], X[0:len(Y)-1, :], missing='drop').fit()
IS_R2 = reg1.rsquared
print("Kitchen Sink OOS R^2 = %6.3f"%(100*OOS_R2))

print('Kitchen Sink Difference in Certainty Equivalence = %7.4f'%(100*(CE-CE_Hist)))

Kitchen Sink OOS R^2 = -12.361
Kitchen Sink Difference in Certainty Equivalence = -0.0368


### Question 3: Create combined forecasts and compute OOS $R^2$ and $\Delta CEV$

In [ ]:
# Create data frame to hold all Y hats for each dependent variable
forecasts = pd.DataFrame()
forecasts[dep_var] = df[dep_var]

for variable in indep_vars:
    Y = np.asarray(df[dep_var])
    X = np.asarray(df[variable])
    Y_hat = np.full(len(Y), np.nan)
    X = sm.add_constant(X)

    for i in range(M+1, len(Y)):
        Y1 = Y[1:i]
        X1 = X[0:i-1, :]
        reg = sm.OLS(Y1, X1, missing='drop').fit()
        Y_hat[i] = np.take(reg.predict(X[i-1, :]), 0)
    forecasts[variable] = Y_hat
forecasts = forecasts.drop(columns=[dep_var])
combined_forecasts = np.asarray(forecasts[indep_vars]) # Transform into an array

# Calculate the median and mean combination forecasts
median_forecast = np.nanmedian(combined_forecasts, axis=1)
mean_forecast = np.nanmean(combined_forecasts, axis=1)

# Calculate out-of-sample R² and ∆CEV for median forecast
OOS_SSE_median = np.sum((Y[M+1:] - median_forecast[M+1:])**2)
OOS_R2_median = 1 - OOS_SSE_median / OOS_SSE_Hist
w1_median = ((1/gam) * (median_forecast / Hist_Variance)).clip(None, 1.5)
r1_median = Y * w1_median
CE_median = np.mean(r1_median[M+1:]) - gam/2 * np.var(r1_median[M+1:], ddof=1)

# Calculate out-of-sample R² and ∆CEV for mean forecast
OOS_SSE_mean = np.sum((Y[M+1:] - mean_forecast[M+1:])**2)
OOS_R2_mean = 1 - OOS_SSE_mean / OOS_SSE_Hist
w1_mean = ((1/gam) * (mean_forecast / Hist_Variance)).clip(None, 1.5)
r1_mean = Y * w1_mean
CE_mean = np.mean(r1_mean[M+1:]) - gam/2 * np.var(r1_mean[M+1:], ddof=1)

# Print results
print("Median Forecast OOS R² = %6.3f" % (100.0 * OOS_R2_median))
print("Median Forecast ∆CEV = %7.4f" % (100 * (CE_median - CE_Hist)))

print("Mean Forecast OOS R² = %6.3f" % (100.0 * OOS_R2_mean))
print("Mean Forecast ∆CEV = %7.4f" % (100 * (CE_mean - CE_Hist)))

Median Forecast OOS R² =  0.574
Median Forecast ∆CEV =  0.0181
Mean Forecast OOS R² =  0.609
Mean Forecast ∆CEV =  0.0019


/var/folders/qn/tkvk2qlj4rg5hmqc92hjphx40000gn/T/ipykernel_18042/2880313194.py:20: RuntimeWarning: All-NaN slice encountered
  median_forecast = np.nanmedian(combined_forecasts, axis=1)
/var/folders/qn/tkvk2qlj4rg5hmqc92hjphx40000gn/T/ipykernel_18042/2880313194.py:21: RuntimeWarning: Mean of empty slice
  mean_forecast = np.nanmean(combined_forecasts, axis=1)
